The goal of this application is to create a **personalized experience** for each buyer, making the **property search process** more engaging and tailored to individual preferences.

The core features of this application are:

1. **Understand Buyer Preference**  
   - Capture Buyer's requirements and preferences, e.g., **location**, **budget**, **property type**, **amenities**, **lifestyle choices**  
   - The application, by use of **LLM**, applies these inputs expressed in natural language in making decisions  

2. **Integration with Vector Database**  
   - All available property listings are stored in a vector database  
   - Vector embeddings are used to match properties with buyer preferences  

3. **Personalized Listing Description Generation**  
   - Highlight the most relevant aspects to buyer preferences when generating listing descriptions of matched properties  
   - Emphasize personalization to make it appealing to the buyer  

4. **Final Presentation**  
   - Final output is presented as a text description of the listing  

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.memory import ConversationSummaryMemory, ConversationBufferMemory, CombinedMemory, ChatMessageHistory
from langchain.chains import ConversationChain
from typing import Any, Dict, Optional, Tuple

from langchain.document_loaders.csv_loader import CSVLoader
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain.chains.question_answering import load_qa_chain



In [10]:
#Step 1: Initialization and setup
import os
import requests
import json
from langchain.llms import OpenAI
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import HumanMessage
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.memory import ConversationSummaryMemory, ConversationBufferMemory, CombinedMemory, ChatMessageHistory
from langchain.chains import ConversationChain
from typing import Any, Dict, Optional, Tuple




os.environ["OPENAI_API_KEY"] = "voc-1591411661266774041819680bf6ddc78900.17113500"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"



In [143]:
#Step 2A: Generating Real Estate Listings

def get_RElistings(l_requests):
    import requests
    import json
    
    structured_listings = []
    limit ="4"
    
    headers = {
        "X-RapidAPI-Key": "134245d733msh6a5a3b56953169cp17e8dajsnbd43f4eef7b0",
        "X-RapidAPI-Host": "realtor16.p.rapidapi.com"
    }
    
    def get_property_details(property_id):
        url = "https://realtor16.p.rapidapi.com/property/details"
        querystring = {"property_id": property_id}
        response = requests.get(url, headers=headers, params=querystring)
        
        if response.status_code == 200:
            data = response.json().get("data", {})
            if data.get("location", {}).get("neighborhoods", {}):
                neighborhood_name = data.get("location", {}).get("neighborhoods", [])[0].get("name", {})
            else:
                neighborhood_name = 'N/A'
            description = data.get("description", {}).get("text", "No description available.")
            no_of_baths = data.get("description", {}).get("baths", "N/A")
            no_of_beds = data.get("description", {}).get("beds", "N/A")
            sqft = data.get("description", {}).get("sqft", "N/A")
            lot_sqft = data.get("description", {}).get("lot_sqft", "N/A")
            
            return neighborhood_name, description, no_of_baths, no_of_beds, sqft, lot_sqft
            
        else:
            #return f"Failed to retrieve data: {response.status_code}"
            return 'N/A', 'No description available.', 'N/A', 'N/A', 'N/A', 'N/A'
            
    def generate_listings(location):
        url = "https://realtor16.p.rapidapi.com/search/forsale"
        querystring = {
            "location": location,
            "limit": limit
        }
        response = requests.get(url, headers=headers, params=querystring)
        listings = response.json().get("properties", [])
        
        if response.status_code == 200:
            data = response.json()
            for listing in listings:
                pid = listing.get("property_id")
                neighborhood_name, description, no_of_baths, no_of_beds, sqft, lot_sqft = get_property_details(pid)
                structured = {
                    "Neighborhood": neighborhood_name,
                    "Address": (
                        f"{listing.get('location', {}).get('address', {}).get('line', '')}, {listing.get('location', {}).get('address', {}).get('city', '')}, {listing.get('location', {}).get('address', {}).get('state_code', '')} {listing.get('location', {}).get('address', {}).get('postal_code', '')}"
                    ),
                    "Postal Code": listing.get('location', {}).get('address', {}).get('postal_code', ''),
                    "Price": f"${listing.get('list_price')}",
                    "Bedrooms": no_of_beds,
                    "Bathrooms": no_of_baths,
                    "House Size": f"{sqft} sqft",
                    "Lot Size": f"{lot_sqft} sqft",
                    "Image": listing.get("primary_photo").get("href"),
                    "Description": description
                }
                structured_listings.append(structured)
            
            return structured_listings
            
        else:
            return f"Failed to retrieve data: {response.status_code}"
            
    #listings_json = generate_listings("scarsdale ny")
    #print(json.dumps(listings_json, indent=4))
    for i in range(len(l_requests)):
        print (l_requests[i])
        generate_listings(l_requests[i])
        
    filename = 'listings.json'
    with open(filename, 'w') as json_file:
        json.dump(structured_listings, json_file, indent=4)  # indent=4 for pretty printing
    print(json.dumps(structured_listings, indent=4))
        

In [144]:
#Step 2B: Storing generated Real Estate Listings in a json file
list_requests = ["scarsdale ny", "white plains ny", "chappaqua ny", "katonah ny", "brooklyn ny"]
get_RElistings(list_requests)

scarsdale ny
white plains ny
chappaqua ny
katonah ny
brooklyn ny
[
    {
        "Neighborhood": "N/A",
        "Address": "135 Montgomery Ave, Scarsdale, NY 10583",
        "Postal Code": "10583",
        "Price": "$425000",
        "Bedrooms": 2,
        "Bathrooms": 1,
        "House Size": "1265 sqft",
        "Lot Size": "4792 sqft",
        "Image": "https://ap.rdcpix.com/e54e30ea3c0b8fc7e5506783fdb1f04cl-m3267675659s.jpg",
        "Description": "Condo Living in Westchester! This spectacular unit is a spacious 2 bed/1 bath updated condo with plenty of storage and parking. Close to all! Short walk to Metro North Train and 32 minutes to NYC. Eastchester Schools. Walk to restaurants, shops, playground and multi-use fields/ green space. Monthly maintenance includes water, sewar, trash, maintenance of common areas, and snow removal. Membership to Lake Isle Pool, Tennis, Golf is available as an Eastchester resident."
    },
    {
        "Neighborhood": "N/A",
        "Address": "105 

In [2]:
#Step 2C: Loading listings json file in a variable
import json

# Open the JSON file and load its content
with open('listings.json', 'r') as file:
    data = json.load(file)

#print(data)
print(json.dumps(data, indent=4))  # Using indent for pretty printing

[
    {
        "Neighborhood": "N/A",
        "Address": "135 Montgomery Ave, Scarsdale, NY 10583",
        "Postal Code": "10583",
        "Price": "$425000",
        "Bedrooms": 2,
        "Bathrooms": 1,
        "House Size": "1265 sqft",
        "Lot Size": "4792 sqft",
        "Image": "https://ap.rdcpix.com/e54e30ea3c0b8fc7e5506783fdb1f04cl-m3267675659s.jpg",
        "Description": "Condo Living in Westchester! This spectacular unit is a spacious 2 bed/1 bath updated condo with plenty of storage and parking. Close to all! Short walk to Metro North Train and 32 minutes to NYC. Eastchester Schools. Walk to restaurants, shops, playground and multi-use fields/ green space. Monthly maintenance includes water, sewar, trash, maintenance of common areas, and snow removal. Membership to Lake Isle Pool, Tennis, Golf is available as an Eastchester resident."
    },
    {
        "Neighborhood": "N/A",
        "Address": "105 Garth 1 Rd Unit 6F and 2, Scarsdale, NY 10583",
        "Postal C

In [3]:
#Step 3: Storing Listings in a Vector Database
##Vector Database Setup
model_name = 'gpt-3.5-turbo'
llm = OpenAI(model_name=model_name, temperature=0, max_tokens=2000)

# prepare the documents from available data
documents = [
    Document(
        page_content=f"Address: {item['Address']}\n\n{item['Description']}",
        metadata={k: v for k, v in item.items() if k != "Description"}
    )
    for item in data
]

# Now safely pass it to the splitter
splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = splitter.split_documents(documents)


# Generate embeddinbg and load into the vector database Chroma
embeddings = OpenAIEmbeddings()

db = Chroma.from_documents(split_docs, embeddings)



/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [4]:
def ask_questions():
    questions = [
        "How big do you want your house to be?",
        "What are 3 most important things for you in choosing this property?",
        "Which amenities would you like?",
        "Which transportation options are important to you?",
        "How urban do you want your neighborhood to be?",   
    ]

    answers = []

    print("Please answer the following questions:\n")

    for question in questions:
        answer = input(f"{question} ")
        answers.append(answer)

    print("\n✅ Data collection complete.\n")

    return questions, answers


In [18]:
## Structure the preferences for querying the vector database
def memoralize_context(personal_questions, personal_answers):
    history = ChatMessageHistory()
    history.add_user_message(f"""You are AI that will recommend user the most appropriate property that is on sale based on their answers to personal questions. Ask user {len(personal_questions)} questions""")
    for i in range(len(personal_questions)):
        history.add_ai_message(personal_questions[i])
        history.add_user_message(personal_answers[i])
    
    history.add_ai_message("""Now tell me a summary of a property you're considering to buy, and specify how you want me to consider when responding""")
    summary_memory = ConversationSummaryMemory(
        llm=llm,
        memory_key="recommendation_summary", 
        input_key="input",
        buffer=f"The human answered {len(personal_questions)} personal questions). Use them to customize the recommended property based on preferences expressed in answering these questions.",
        return_messages=True)

    # you could choose to store some of the q/a in memory as well, in addition to original questions
    class MementoBufferMemory(ConversationBufferMemory):
        def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
            input_str, output_str = self._get_input_output(inputs, outputs)
            self.chat_memory.add_ai_message(output_str)
    
    conversational_memory = MementoBufferMemory(
        chat_memory=history,
        memory_key="questions_and_answers", 
        input_key="input"
    )

    # Combined
    memory = CombinedMemory(memories=[conversational_memory, summary_memory])
        
    messages = conversational_memory.chat_memory.messages
    user_inputs = [msg.content for msg in messages if isinstance(msg, HumanMessage)]
    user_text = "\n".join(user_inputs)
    user_pref = llm.invoke(f"Summarize the following user inputs:\n{user_text}")
    
    return user_pref, memory


In [19]:
def extract_similar_documents(query):
    similar_docs = db.similarity_search(query, k=5)
    return similar_docs


In [29]:
def recommend_listings(similar_docs, personal_questions, personal_answers, memory):
    
    max_rating = 10
    

    RECOMMENDER_TEMPLATE = """The following is a friendly conversation between a human and an AI Movie Recommender. 
                            The AI follows human instructions and provides prefrred  property on sale for a human based on property details
                            and human's persona derived from their answers to questions. 

    Summary of Recommendations:
    {recommendation_summary}
    Personal Questions and Answers:
    {questions_and_answers}
    Human: {input}
    AI:"""
    PROMPT = PromptTemplate(
        input_variables=["recommendation_summary", "input", "questions_and_answers"],
        template=RECOMMENDER_TEMPLATE
    )
    recommender = ConversationChain(llm=llm, verbose=True, memory=memory, prompt=PROMPT)
 
    for property in similar_docs:
    
        property_address = property.metadata['Address']
        no_of_beds = property.metadata['Bedrooms']
        no_of_baths = property.metadata['Bathrooms']
        list_price = property.metadata['Price']
        property_details = property.page_content
    
        property_selection_instructions = f"""
             =====================================
            === START PROPERTY DEATILS SUMMARY FOR {property_address} ===
            {property_details}
            === END PROPERTY DEATILS SUMMARY ===
            =====================================
        
            RATING INSTRUCTIONS THAT MUST BE STRICTLY FOLLOWED:
            AI will provide a highly personalized rating based only on the property details summary human provided 
            and human answers to questions included with the context. 
            AI should be highly cognizant to human personal preferences captured in the answers to personal questions, 
            and should not be influenced by anything else.
            AI will also build a persona for human based on human answers to questions, and use this persona to rate the property.
            OUTPUT FORMAT:
            First, include that persona you came up with in the explanation for the rating. Describe the persona in a few sentences.
            Explain how human preferences captured in the answers to personal questions influenced creation of this persona.
            In addition, consider other attributes for this human that you might have deciphered as they might give you more information about human's preferences.
            Your goal is to provide a rating based on how closely the details of this property matches to preferences that human has exhibited.
            Remember that human has very limited time and would like to see property they will like, so your rating should be as accurate as possible.
            Rating will range from 1 to {max_rating}, with {max_rating} meaning human will love it, and 1 meaning human will hate it. 
            In providing the rating consider how close {no_of_beds} bedrooms, {no_of_baths} bathrooms and list price {list_price} if the human has specified in answers to questions. 
            You will include a logical explanation for your rating based on human persona you've build and human responses to questions.
            YOUR REVIEW MUST END WITH TEXT: "RATING FOR ADDRESS {property_address} is " FOLLOWED BY THE RATING.
            FOLLOW THE INSTRUCTIONS STRICTLY, OTHERWISE HUMAN WILL NOT BE ABLE TO TAKE INTO CONSIDERATION YOUR REVIEW IN THE DECISION PROCESS.
        """
        prediction = recommender.predict(input=property_selection_instructions)
        print(prediction)
        
    final_recommendation = """Now that AI has assessed all the properties, AI will recommend human the one that human will like the most. 
                                AI will respond with property recommendation, and short explanation for why human will like it over all other properties. 
                                AI will not include any ratings in your explanation, only the reasons why human will like it the most.
                                However, the property you will pick must be one of the properties you rated the highest.
                                For example, if you rated one property 8, and the other 7, you will recommend the property with rating 8 because rating 8 
                                is greater than rating of 7."""

    prediction = recommender.predict(input=final_recommendation)
    print(prediction)

In [30]:
def get_property_recommendation(personal_questions, personal_answers):
    
    #personal_questions, personal_answers = ask_questions()
    user_pref, memory =  memoralize_context(personal_questions, personal_answers)
    similar_docs = extract_similar_documents(user_pref)
    recommend_listings(similar_docs, personal_questions, personal_answers, memory)
    
    return
    


In [32]:
#Test Case 1

personal_questions = [   
                "How big do you want your house to be?" 
                "What are 3 most important things for you in choosing this property?", 
                "Which amenities would you like?", 
                "Which transportation options are important to you?",
                "How urban do you want your neighborhood to be?",   
            ]
personal_answers = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."]

get_property_recommendation(personal_questions, personal_answers)




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI Movie Recommender. 
                            The AI follows human instructions and provides prefrred  property on sale for a human based on property details
                            and human's persona derived from their answers to questions. 

    Summary of Recommendations:
    [SystemMessage(content='The human answered 4 personal questions). Use them to customize the recommended property based on preferences expressed in answering these questions.')]
    Personal Questions and Answers:
    Human: You are AI that will recommend user the most appropriate property that is on sale based on their answers to personal questions. Ask user 4 questions
AI: How big do you want your house to be?What are 3 most important things for you in choosing this property?
Human: A comfortable three-bedroom house with a spacious kitchen and a cozy living room.
AI: Wh


> Finished chain.
Based on the property details summary you provided for 200 Millwood Rd, Chappaqua, NY 10514, this reimagined farmhouse seems to align well with your preferences for a peaceful and private retreat. The 4-bedroom layout offers ample space for family and guests, while the modern updates and high-end finishes in the kitchen cater to your desire for comfort and functionality. The serene setting with mature landscaping, ponds, and wide-open views provides the privacy and outdoor space you value. The detached garage with finished space above also offers versatility for various uses like a studio, gym, or guest suite.

Considering the spacious layout, modern amenities, and private setting of this farmhouse property, I would rate it a 9 out of 10 for you. The property seems to meet your criteria exceptionally well, offering a blend of timeless charm and modern comfort in a peaceful and secluded environment.

RATING FOR ADDRESS 200 Millwood Rd, Chappaqua, NY 10514 is 9.


> En


> Finished chain.
Based on the property details summary you provided for 249 Mill River Rd, Chappaqua, NY 10514, the contemporary bi-level 4-bedroom, 2-bathroom home seems to offer a blend of comfort and sophistication. The recent upgrades like a new roof, boiler, and sanitary system indicate a focus on modern amenities and functionality. The large living room and three freshly stained multi-level decks cater to your desire for ample space for gatherings and outdoor entertainment. The large yard with three sheds provides plenty of outdoor storage and space for activities.

Considering your preference for comfort, space, and practical features in a home, this property aligns well with your criteria. The location within the prestigious Pleasantville School system and the convenience of a free school bus pick up in a walking school district add to the appeal of the property. The proximity to the Chappaqua Train station for easy access to Grand Central Station also enhances the property's


> Finished chain.
Based on the property details summary you provided for 7007 Narrows Ave, Brooklyn, NY 11209, the tastefully renovated semi-detached home seems to offer approximately 1,900 square feet of interior space across three levels, with 3 bedrooms and 3 bathrooms. The open living and dining area, updated eat-in kitchen, and hardwood floors throughout cater to your preference for comfort and functionality. The generous closet space in all bedrooms, including multiple walk-in closets, aligns well with your desire for ample storage. The fully finished basement with a separate entrance, large room, full bathroom, and laundry room offers additional living space and convenience.

Considering your persona as someone who values comfort, space, and privacy in a home, the property details seem to align well with your preferences. The proximity to parks, waterfront paths, shopping, and public transportation also meets your criteria for a convenient location. The shared driveway, detache


> Finished chain.
Based on the property details summary you provided for 105 Garth 1 Rd Unit 6F and 2, Scarsdale, NY 10583, the beautifully renovated and generously sized 3-bedroom, 3-full bath co-op seems to align well with your preferences for comfort, space, and luxury. The tasteful upgrades, spacious layout, and low-maintenance living offer the ease and elegance you value in a home. The convenient location near Scarsdale Metro North, bus stops, eateries, shops, and private parking options cater to your desire for modern convenience and accessibility.

Considering your persona as someone who values tranquility, stress-free living, and convenience, this property seems to meet your criteria exceptionally well. The abundance of storage space, fully renovated bathrooms, luxurious kitchen, and upgraded features like Sonos surround sound system enhance the overall appeal of the property.

Taking into account the 3 bedrooms, 3 bathrooms, and list price of $799,000, I would rate this prope


> Finished chain.
Based on the information provided and your preferences for comfort, space, and privacy in a home, I would recommend the property at 341 Millwood Rd, Chappaqua, NY 10514. This property aligns exceptionally well with your criteria, offering a spacious kitchen, cozy living room, ample storage space, privacy, and proximity to good local schools and shopping options. The 4 bedrooms, 3 bathrooms, and list price of $899,000 make this property a highly suitable option for you. The serene and family-friendly environment of this property seems to cater to your desires and priorities, making it the top recommendation among the properties assessed.


In [34]:
#Test Case 2

personal_questions = [   
                "How big do you want your house to be?" 
                "What are 3 most important things for you in choosing this property?", 
                "Which amenities would you like?", 
                "Which transportation options are important to you?",
                "How urban do you want your neighborhood to be?",   
            ]
personal_answers = [
    "A comfortable two-bedroom apartment for me and my spouse. Not interested in more than 3-bedroom",
    "A bustling city experience, public transportation and convenient shopping options.",
    "A fitness center nearby, common areas, recreational spaces.",
    "Easy access to a reliable bus line, public transportaation, and walking pavements",
    "A safe city life with access to amenities like restaurants and theaters."]

get_property_recommendation(personal_questions, personal_answers)




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI Movie Recommender. 
                            The AI follows human instructions and provides prefrred  property on sale for a human based on property details
                            and human's persona derived from their answers to questions. 

    Summary of Recommendations:
    [SystemMessage(content='The human answered 4 personal questions). Use them to customize the recommended property based on preferences expressed in answering these questions.')]
    Personal Questions and Answers:
    Human: You are AI that will recommend user the most appropriate property that is on sale based on their answers to personal questions. Ask user 4 questions
AI: How big do you want your house to be?What are 3 most important things for you in choosing this property?
Human: A comfortable two-bedroom apartment for me and my spouse. Not interested in more than 3-


> Finished chain.
Based on the personal questions and answers provided by the human, I have built a persona for them. The human is looking for a comfortable two-bedroom apartment in a bustling city with easy access to public transportation and convenient shopping options. They also value amenities such as a fitness center nearby, common areas, and recreational spaces. 

The property details for 7007 Narrows Ave in Brooklyn, NY 11209 do not align well with the human's preferences. While the property offers 3 bedrooms and 3 bathrooms, it is a semi-detached home with a different layout and features compared to the cozy apartment in a bustling city that the human is looking for. Additionally, the list price of $1,395,000 may be higher than what the human is willing to spend on a property.

Considering the mismatch between the property details and the human's preferences, I would rate this property a 4 out of 10. The property does not meet the criteria of a comfortable two-bedroom apartmen


> Finished chain.
Based on the personal questions and answers provided by the human, I have built a persona for them. The human is looking for a comfortable two-bedroom apartment in a bustling city with easy access to public transportation and convenient shopping options. They also value amenities such as a fitness center nearby, common areas, and recreational spaces. The human prefers a cozy and well-maintained living space with updated features.

The property details for 505 Central Ave Apt 618 in White Plains, NY 10606 seem to align well with the human's preferences. This beautifully maintained 2 bedroom/1 bath unit offers updated kitchen and bath, hardwood floors, custom storage in the master bedroom, and a balcony with wooded views. The building includes amenities such as parking, elevators, storage room, and on-site super. The location is close to transportation options, schools, parks, and shopping.

Considering the property details and the human's persona, I would rate this pr


> Finished chain.
Based on the personal questions and answers provided by the human, I have built a persona for them. The human is looking for a comfortable two-bedroom apartment in a bustling city with easy access to public transportation and convenient shopping options. They also value amenities such as a fitness center nearby, common areas, and recreational spaces. The human prefers a cozy and well-maintained living space with updated features.

The property details for 281 Garth Rd Apt B5D in Scarsdale, NY 10583 do not align well with the human's preferences. This charming studio co-op in Garth Manor offers western views overlooking a manicured courtyard, a dressing room, three large closets, dining foyer, and parquet floors. While the location is convenient with access to Scarsdale Village, Metro North Train, and restaurants, the studio layout and lack of a second bedroom may not meet the human's criteria for a two-bedroom apartment. Additionally, the maintenance fees and restric


> Finished chain.
Based on the personal questions and answers provided by the human, I have built a persona for them. The human is looking for a comfortable two-bedroom apartment in a bustling city with easy access to public transportation and convenient shopping options. They also value amenities such as a fitness center nearby, common areas, and recreational spaces. The human prefers a cozy and well-maintained living space with updated features.

The property details for 341 Millwood Rd in Chappaqua, NY 10514 do not align well with the human's preferences. This 4-bedroom, 2.5-bath home on nearly 2 acres of wooded land offers space, comfort, and privacy, which may not match the human's criteria for a two-bedroom apartment in a bustling city. The list price of $899,000 may also be higher than what the human is willing to spend on a property.

Considering the mismatch between the property details and the human's preferences, I would rate this property a 2 out of 10. The property does n


> Finished chain.
Based on the personal questions and answers provided by the human, the property that aligns best with your preferences is 50 Dekalb Ave Apt S10 in White Plains, NY 10605. This property offers a cozy fireplace, breezy balcony, assigned parking for 2 cars, and is located in a bustling city with easy access to public transportation and convenient shopping options. The amenities and features of this property make it highly appealing to you based on your persona and preferences. Enjoy the comfortable two-bedroom apartment in a vibrant city setting at 50 Dekalb Ave Apt S10!


In [ ]:
RECOMMENDER_TEMPLATE = """The following is a friendly conversation between a human and an AI Movie Recommender. 
                        The AI follows human instructions and provides prefrred  property on sale for a human based on property details
                        and human's persona derived from their answers to questions. 

Summary of Recommendations:
{recommendation_summary}
Personal Questions and Answers:
{questions_and_answers}
Human: {input}
AI:"""
PROMPT = PromptTemplate(
    input_variables=["recommendation_summary", "input", "questions_and_answers"],
    template=RECOMMENDER_TEMPLATE
)
recommender = ConversationChain(llm=llm, verbose=True, memory=memory, prompt=PROMPT)
 
for property in similar_docs:
    
    property_address = property.metadata['Address']
    no_of_beds = property.metadata['Bedrooms']
    no_of_baths = property.metadata['Bathrooms']
    list_price = property.metadata['Price']
    property_details = property.page_content
    
    property_selection_instructions = f"""
         =====================================
        === START PROPERTY DEATILS SUMMARY FOR {property_address} ===
        {property_details}
        === END PROPERTY DEATILS SUMMARY ===
        =====================================
        
        RATING INSTRUCTIONS THAT MUST BE STRICTLY FOLLOWED:
        AI will provide a highly personalized rating based only on the property details summary human provided 
        and human answers to questions included with the context. 
        AI should be highly cognizant to human personal preferences captured in the answers to personal questions, 
        and should not be influenced by anything else.
        AI will also build a persona for human based on human answers to questions, and use this persona to rate the property.
        OUTPUT FORMAT:
        First, include that persona you came up with in the explanation for the rating. Describe the persona in a few sentences.
        Explain how human preferences captured in the answers to personal questions influenced creation of this persona.
        In addition, consider other attributes for this human that you might have deciphered as they might give you more information about human's preferences.
        Your goal is to provide a rating based on how closely the details of this property matches to preferences that human has exhibited.
        Remember that human has very limited time and would like to see property they will like, so your rating should be as accurate as possible.
        Rating will range from 1 to {max_rating}, with {max_rating} meaning human will love it, and 1 meaning human will hate it. 
        In providing the rating consider how close {no_of_beds} bedrooms, {no_of_baths} bathrooms and list price {list_price} if the human has specified in answers to questions. 
        You will include a logical explanation for your rating based on human persona you've build and human responses to questions.
        YOUR REVIEW MUST END WITH TEXT: "RATING FOR ADDRESS {property_address} is " FOLLOWED BY THE RATING.
        FOLLOW THE INSTRUCTIONS STRICTLY, OTHERWISE HUMAN WILL NOT BE ABLE TO TAKE INTO CONSIDERATION YOUR REVIEW IN THE DECISION PROCESS.
    """
    prediction = recommender.predict(input=property_selection_instructions)
    print(prediction)


In [3]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain.chains.question_answering import load_qa_chain

import os

os.environ["OPENAI_API_KEY"] = "voc-1591411661266774041819680bf6ddc78900.17113500"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

# print(docs)
model_name = 'gpt-3.5-turbo'
llm = OpenAI(model_name=model_name, temperature=0, max_tokens=2000)

from langchain.schema import Document

# Assuming your list is called `data`
documents = [
    Document(
        page_content=f"Address: {item['Address']}\n\n{item['Description']}",
        metadata={k: v for k, v in item.items() if k != "Description"}
    )
    for item in data
]

# Now you can safely pass it to the splitter
splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = splitter.split_documents(documents)



embeddings = OpenAIEmbeddings()

db = Chroma.from_documents(split_docs, embeddings)
query = """
    Based on the list of similar listings that are available for sale in the context, tell me what I should choose
    considering that my faamily consists of myself, my wife and two daughters. A good school is important to me and 
    away from hustle and bustle of city. Please also list the details of the property you are recommending. Also print the 
    metadata for this property.
    """

use_chain_helper = False
if use_chain_helper:
    rag = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever())
    print(rag.run(query))
else:
    similar_docs = db.similarity_search(query, k=5)
    prompt = PromptTemplate(
        template="{query}\nContext: {context}",
        input_variables=["query", "context"],
    )
    chain = load_qa_chain(llm, prompt = prompt, chain_type="stuff")
    print(chain.run(input_documents=similar_docs, query = query))

Based on your preferences for a good school and a peaceful location away from the city, I recommend the reimagined farmhouse on a 3+ acre private setting. This 4-bedroom home offers modern comfort with timeless charm, featuring wide-plank hardwood floors, a beautifully updated kitchen, and a serene primary suite with a spa-inspired bath. The property includes mature landscaping, two tranquil ponds, and wide-open views, providing a peaceful retreat for your family.

Property Details:
- Address: Not provided in the context
- Bedrooms: 4
- Bathrooms: Not specified
- Lot Size: 3+ acres
- Features: Wide-plank hardwood floors, updated kitchen, serene primary suite, mature landscaping, two tranquil ponds

Metadata:
- Listing Price: Not provided in the context
- Year Built: Not provided in the context
- Square Footage: Not provided in the context
- School District: Not provided in the context
- Property Type: Farmhouse
- Amenities: Detached garage with finished space above, dedicated office, p

In [23]:
print(similar_docs[0])
#for property in similar_docs:
#    print (property.metadata['Address'])
#    print (property.page_content)

page_content="Nestled on nearly 2 acres of serene, wooded land, this beautifully maintained 4-bedroom, 2.5-bath home offers space, comfort, and privacy. The bright and airy eat-in kitchen features abundant cabinet space, a convenient in-kitchen laundry, and a large picture window overlooking the scenic backyard. A sun-filled living room with an oversized picture window enhances the home's warm and inviting atmosphere. The generously sized bedrooms provide ample space for relaxation, while the renovated lower level boasts all-tile flooring, a cozy wood-burning fireplace in the family room, an additional bedroom, a powder room, and plenty of storage. Enjoy outdoor living on the spacious deck, perfect for entertaining or unwinding while taking in the tranquil surroundings. The upper level of the private yard offers even more space, perfect for a fire pit or additional outdoor activities. Additional highlights include a full-house generator for year-round peace of mind, freshly painted int

In [7]:
from langchain.memory import ConversationSummaryMemory, ConversationBufferMemory, CombinedMemory, ChatMessageHistory
personal_questions = [   
                "How big do you want your house to be?" 
                "What are 3 most important things for you in choosing this property?", 
                "Which amenities would you like?", 
                "Which transportation options are important to you?",
                "How urban do you want your neighborhood to be?",   
            ]
personal_answers = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."]


## Structure the preferences for querying the vector database
history = ChatMessageHistory()
history.add_user_message(f"""You are AI that will recommend user the most appropriate property that is on sale based on their answers to personal questions. Ask user {len(personal_questions)} questions""")
# TODO: add questions and answers to the history
for i in range(len(personal_questions)):
    history.add_ai_message(personal_questions[i])
    history.add_user_message(personal_answers[i])

    
print(history)



messages=[HumanMessage(content='You are AI that will recommend user the most appropriate property that is on sale based on their answers to personal questions. Ask user 4 questions'), AIMessage(content='How big do you want your house to be?What are 3 most important things for you in choosing this property?'), HumanMessage(content='A comfortable three-bedroom house with a spacious kitchen and a cozy living room.'), AIMessage(content='Which amenities would you like?'), HumanMessage(content='A quiet neighborhood, good local schools, and convenient shopping options.'), AIMessage(content='Which transportation options are important to you?'), HumanMessage(content='A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.'), AIMessage(content='How urban do you want your neighborhood to be?'), HumanMessage(content='Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.')]


In [15]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain.chains.question_answering import load_qa_chain

import os

os.environ["OPENAI_API_KEY"] = "voc-1591411661266774041819680bf6ddc78900.17113500"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

# print(docs)
model_name = 'gpt-3.5-turbo'
llm = OpenAI(model_name=model_name, temperature=0, max_tokens=2000)

from langchain.schema import Document

# Assuming your list is called `data`
documents = [
    Document(
        page_content=f"Address: {item['Address']}\n\n{item['Description']}",
        metadata={k: v for k, v in item.items() if k != "Description"}
    )
    for item in data
]

# Now you can safely pass it to the splitter
splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = splitter.split_documents(documents)



embeddings = OpenAIEmbeddings()

db = Chroma.from_documents(split_docs, embeddings)
query = """
    Based on the list of similar listings that are available for sale in the context, tell me what I should choose
    considering that my faamily consists of myself, my wife and two daughters. A good school is important to me and 
    away from hustle and bustle of city. Please also list the details of the property you are recommending. Also print the 
    metadata for this property.
    """

use_chain_helper = False
if use_chain_helper:
    rag = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever())
    print(rag.run(query))
else:
    similar_docs = db.similarity_search(query, k=5)
    prompt = PromptTemplate(
        template="{query}\nContext: {context}",
        input_variables=["query", "context"],
    )
    chain = load_qa_chain(llm, prompt = prompt, chain_type="stuff")
    print(chain.run(input_documents=similar_docs, query = query))

page_content='Detach 4 family handyman short sale property subject to third party approval needs total renovation needs interior work, plumbing and electrical off of Church Ave close to Hospital and public transportation has lot of potential, Property has private driveway' metadata={'Address': '494 E 96th St, Brooklyn, NY 11212', 'Bathrooms': 4, 'Bedrooms': 16, 'House Size': 'None sqft', 'Image': 'https://ap.rdcpix.com/00bf0d99aff13732a0687258b6a1f73dl-m3092275580s.jpg', 'Lot Size': 'None sqft', 'Neighborhood': 'West Waverly', 'Postal Code': '11212', 'Price': '$500000'}


In [ ]:
personal_questions = [   
                "How big do you want your house to be?" 
                "What are 3 most important things for you in choosing this property?", 
                "Which amenities would you like?", 
                "Which transportation options are important to you?",
                "How urban do you want your neighborhood to be?",   
            ]
personal_answers = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."]

In [ ]:
## Structure the preferences for querying the vector database
history = ChatMessageHistory()
history.add_user_message(f"""You are AI that will recommend user the most appropriate property that is on sale based on their answers to personal questions. Ask user {len(personal_questions)} questions""")
# TODO: add questions and answers to the history
for i in range(len(personal_questions)):
    history.add_ai_message(personal_questions[i])
    history.add_user_message(personal_answers[i])
    
    
history.add_ai_message("""Now tell me a plot summary of a movie you're considering watching, and specify how you want me to respond to you with the movie rating""")
summary_memory = ConversationSummaryMemory(
    llm = llm,
    memory_key = "recommendation_summary",
    input_key = "input",
    #buffer = f"The human answered {len(personal_questions)} personal questions. Use them to rate, from 1 to 5",
    buffer=f"The human answered {len(personal_questions)} personal questions). Use them to rate, from 1 to {max_rating}, how much they like a movie they describe to you.",
    return_messages = True

)

In [4]:
#Step 3: Storing Listings in a Vector Database
##Vector Database Setup
# TODO: initialize your embeddings model
embeddings = OpenAIEmbeddings()
# TODO: populate your vector database with the chunks
db = Chroma.from_documents(split_docs, embeddings)

##Generating and Soting EDmbeddings
# find top 5 semantically similar documents to the query
top_5 = db.similarity_search(query, k=5)
print(top_5)

In [ ]:
use_chain_helper = False
if use_chain_helper:
    rag = RetrievalQA.from_chain_type(llm = llm, chain_type="stuff", retriever = db.as_retriever())
    print(rag.run(query))
else:
    top_5 = db.similarity_search(query, k=5)
    prompt = PromptTemplate(
        template = "{query}\nContext = {context}     ",
        input_variables = ["query", "context"],
    )
    chain = load_qa_chain(llm, prompt = prompt, chain_type="stuff")
    response = chain.run(input_documents=top_5, query=query)
    print(response)
    
    

In [1]:
#Step 4: Build the User Preference Interface
## Capture User Preferences
# update these questions as you think will be the most helpful for your AI recommender
personal_questions = [  "Which movie genre you like the most?", 
                        "What is your favorite color?", 
                        "What is your favorite movie?", 
                        "Pick one - dogs, cats or hamsters?",
                        "What is your favorite food?",
                        "What is your favorite drink?" ]

    
# list of your personal answers to the questions above
personal_answers = ['thriller', 'blue', 'inception', 'dogs', 'fish tacos', 'strawberry milkshake']


## Structure the preferences for querying the vector database
history = ChatMessageHistory()
history.add_user_message(f"""You are AI that will recommend user a movie based on their answers to personal questions. Ask user {len(personal_questions)} questions""")
# TODO: add questions and answers to the history
for i in range(len(personal_questions)):
    history.add_ai_message(personal_questions[i])
    history.add_user_message(personal_answers[i])
    
    
history.add_ai_message("""Now tell me a plot summary of a movie you're considering watching, and specify how you want me to respond to you with the movie rating""")
summary_memory = ConversationSummaryMemory(
    llm = llm,
    memory_key = "recommendation_summary",
    input_key = "input",
    #buffer = f"The human answered {len(personal_questions)} personal questions. Use them to rate, from 1 to 5",
    buffer=f"The human answered {len(personal_questions)} personal questions). Use them to rate, from 1 to {max_rating}, how much they like a movie they describe to you.",
    return_messages = True

)

In [2]:
#Step 5: Search listing Based on Preferences
## Semantic seaarch with user prefernces to retrive listings that matches closely

## Fine tune to retrieve the most relevant listings

In [3]:
# Step 6: Personalize Listing Descriptions
## LLM Augmentation to generate tailor made descriptions aligned to buyer perefernces



In [4]:
# Step 7: Test for different buyer preferences

In [35]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.memory import ConversationSummaryMemory, ConversationBufferMemory, CombinedMemory, ChatMessageHistory
from langchain.chains import ConversationChain
from typing import Any, Dict, Optional, Tuple

import requests
import os

os.environ["OPENAI_API_KEY"] = "voc-1591411661266774041819680bf6ddc78900.17113500"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"


model_name = "gpt-3.5-turbo"
temperature = 0.0
llm = OpenAI(model_name=model_name, temperature=temperature, max_tokens = 1000)

from langchain.memory import ConversationSummaryMemory, ConversationBufferMemory, CombinedMemory, ChatMessageHistory
personal_questions = [   
                "How big do you want your house to be?" 
                "What are 3 most important things for you in choosing this property?", 
                "Which amenities would you like?", 
                "Which transportation options are important to you?",
                "How urban do you want your neighborhood to be?",   
            ]
personal_answers = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."]

max_rating = 10

## Structure the preferences for querying the vector database
history = ChatMessageHistory()
history.add_user_message(f"""You are AI that will recommend user the most appropriate property that is on sale based on their answers to personal questions. Ask user {len(personal_questions)} questions""")
for i in range(len(personal_questions)):
    history.add_ai_message(personal_questions[i])
    history.add_user_message(personal_answers[i])
    
history.add_ai_message("""Now tell me a summary of a property you're considering to buy, and specify how you want me to consider when responding""")
summary_memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="recommendation_summary", 
    input_key="input",
    buffer=f"The human answered {len(personal_questions)} personal questions). Use them to customize the recommended property based on preferences expressed in answering these questions.",
    return_messages=True)

# you could choose to store some of the q/a in memory as well, in addition to original questions
class MementoBufferMemory(ConversationBufferMemory):
    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        input_str, output_str = self._get_input_output(inputs, outputs)
        self.chat_memory.add_ai_message(output_str)
    
conversational_memory = MementoBufferMemory(
    chat_memory=history,
    memory_key="questions_and_answers", 
    input_key="input"
)

# Combined
memory = CombinedMemory(memories=[conversational_memory, summary_memory])
RECOMMENDER_TEMPLATE = """The following is a friendly conversation between a human and an AI Movie Recommender. 
                        The AI follows human instructions and provides prefrred  property on sale for a human based on property details
                        and human's persona derived from their answers to questions. 

Summary of Recommendations:
{recommendation_summary}
Personal Questions and Answers:
{questions_and_answers}
Human: {input}
AI:"""
PROMPT = PromptTemplate(
    input_variables=["recommendation_summary", "input", "questions_and_answers"],
    template=RECOMMENDER_TEMPLATE
)
recommender = ConversationChain(llm=llm, verbose=True, memory=memory, prompt=PROMPT)
 
for property in similar_docs:
    
    property_address = property.metadata['Address']
    no_of_beds = property.metadata['Bedrooms']
    no_of_baths = property.metadata['Bathrooms']
    list_price = property.metadata['Price']
    property_details = property.page_content
    
    property_selection_instructions = f"""
         =====================================
        === START PROPERTY DEATILS SUMMARY FOR {property_address} ===
        {property_details}
        === END PROPERTY DEATILS SUMMARY ===
        =====================================
        
        RATING INSTRUCTIONS THAT MUST BE STRICTLY FOLLOWED:
        AI will provide a highly personalized rating based only on the property details summary human provided 
        and human answers to questions included with the context. 
        AI should be highly cognizant to human personal preferences captured in the answers to personal questions, 
        and should not be influenced by anything else.
        AI will also build a persona for human based on human answers to questions, and use this persona to rate the property.
        OUTPUT FORMAT:
        First, include that persona you came up with in the explanation for the rating. Describe the persona in a few sentences.
        Explain how human preferences captured in the answers to personal questions influenced creation of this persona.
        In addition, consider other attributes for this human that you might have deciphered as they might give you more information about human's preferences.
        Your goal is to provide a rating based on how closely the details of this property matches to preferences that human has exhibited.
        Remember that human has very limited time and would like to see property they will like, so your rating should be as accurate as possible.
        Rating will range from 1 to {max_rating}, with {max_rating} meaning human will love it, and 1 meaning human will hate it. 
        In providing the rating consider how close {no_of_beds} bedrooms, {no_of_baths} bathrooms and list price {list_price} if the human has specified in answers to questions. 
        You will include a logical explanation for your rating based on human persona you've build and human responses to questions.
        YOUR REVIEW MUST END WITH TEXT: "RATING FOR ADDRESS {property_address} is " FOLLOWED BY THE RATING.
        FOLLOW THE INSTRUCTIONS STRICTLY, OTHERWISE HUMAN WILL NOT BE ABLE TO TAKE INTO CONSIDERATION YOUR REVIEW IN THE DECISION PROCESS.
    """
    prediction = recommender.predict(input=property_selection_instructions)
    print(prediction)

final_recommendation = """Now that AI has assessed all the properties, AI will recommend human the one that human will like the most. 
                            AI will respond with property recommendation, and short explanation for why human will like it over all other properties. 
                            AI will not include any ratings in your explanation, only the reasons why human will like it the most.
                            However, the property you will pick must be one of the properties you rated the highest.
                            For example, if you rated one property 8, and the other 7, you will recommend the property with rating 8 because rating 8 
                            is greater than rating of 7."""

prediction = recommender.predict(input=final_recommendation)
print(prediction)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI Movie Recommender. 
                        The AI follows human instructions and provides prefrred  property on sale for a human based on property details
                        and human's persona derived from their answers to questions. 

Summary of Recommendations:
[SystemMessage(content='The human answered 4 personal questions). Use them to customize the recommended property based on preferences expressed in answering these questions.')]
Personal Questions and Answers:
Human: You are AI that will recommend user the most appropriate property that is on sale based on their answers to personal questions. Ask user 4 questions
AI: How big do you want your house to be?What are 3 most important things for you in choosing this property?
Human: A comfortable three-bedroom house with a spacious kitchen and a cozy living room.
AI: Which amenities would you 


> Finished chain.
Based on the personal questions you answered, it seems like you value comfort, space, and privacy in a property. Your preference for a comfortable three-bedroom house with a spacious kitchen and a cozy living room aligns well with the property details you provided. The property at 341 Millwood Rd offers 4 bedrooms and 2.5 bathrooms, which meets your desired bedroom count. The spacious kitchen with abundant cabinet space and the cozy living room with an oversized picture window seem to match your preferences for a comfortable living space. Additionally, the serene, wooded land and the spacious deck for outdoor living align with your desire for privacy and outdoor space. Considering all these factors, I would rate this property a 9 out of 10 for you.

RATING FOR ADDRESS 341 Millwood Rd, Chappaqua, NY 10514 is 9


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI Movie Recommender. 
    


> Finished chain.
Based on the personal questions you answered, it seems like you value comfort, space, and privacy in a property. Your preference for a comfortable three-bedroom house with a spacious kitchen and a cozy living room aligns well with the property details you provided for 200 Millwood Rd. The property offers 4 bedrooms and 3 bathrooms, exceeding your desired bedroom count and providing an additional bathroom. The open, sunlit layout with oversized windows and wide-plank hardwood floors seems to match your preferences for a comfortable living space. The beautifully updated kitchen with high-end finishes and seamless flow into dining and gathering areas align with your desire for a spacious kitchen. The serene primary suite with a spa-inspired bath and large walk-in closet offers a private retreat, meeting your preference for privacy. The additional bedrooms and dedicated office provide ample space for family, guests, or working from home, catering to your need for space. 


> Finished chain.
Based on the property details you provided for 249 Mill River Rd, Chappaqua, NY 10514, it seems to offer a contemporary bi-level 4-bedroom, 2-bathroom home with red oak hardwood flooring and recent upgrades. The large living room and multiple decks for entertaining align with your preference for comfort and space. The large yard with outdoor storage and park-like grounds seem to cater to your desire for outdoor activities and privacy. The property's location within the prestigious Pleasantville School system and proximity to the Chappaqua Train station offer convenience and easy access to transportation options. Considering your preferences for comfort, space, and privacy, as well as the property details provided, I would rate this property a 8 out of 10 for you.

RATING FOR ADDRESS 249 Mill River Rd, Chappaqua, NY 10514 is 8


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI Movie R


> Finished chain.
Based on the analysis of the properties and your personal preferences for comfort, space, and privacy, I recommend the property at 200 Millwood Rd. This property offers a comfortable living space with 4 bedrooms and 3 bathrooms, exceeding your desired bedroom count and providing an additional bathroom. The open, sunlit layout with oversized windows and wide-plank hardwood floors aligns with your preference for a comfortable living space. The beautifully updated kitchen with high-end finishes and seamless flow into dining and gathering areas caters to your desire for a spacious kitchen. The serene primary suite with a spa-inspired bath and large walk-in closet offers a private retreat, meeting your preference for privacy. The additional bedrooms and dedicated office provide ample space for family, guests, or working from home, fulfilling your need for space. The detached garage with finished space above offers versatility for various uses, aligning with your desire fo

In [31]:
messages = conversational_memory.chat_memory.messages
from langchain.schema import HumanMessage

user_inputs = [msg.content for msg in messages if isinstance(msg, HumanMessage)]
user_text = "\n".join(user_inputs)
from langchain.chains.summarize import load_summarize_chain

user_pref = llm.invoke(f"Summarize the following user inputs:\n{user_text}")
print(user_pref)



User is looking for a comfortable three-bedroom house with a spacious kitchen and cozy living room in a quiet neighborhood with good local schools and convenient shopping options. They also want a backyard for gardening, a two-car garage, and a modern, energy-efficient heating system. Additionally, they prefer easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.


In [32]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain.chains.question_answering import load_qa_chain

import os

os.environ["OPENAI_API_KEY"] = "voc-1591411661266774041819680bf6ddc78900.17113500"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

# print(docs)
model_name = 'gpt-3.5-turbo'
llm = OpenAI(model_name=model_name, temperature=0, max_tokens=2000)

from langchain.schema import Document

# Assuming your list is called `data`
documents = [
    Document(
        page_content=f"Address: {item['Address']}\n\n{item['Description']}",
        metadata={k: v for k, v in item.items() if k != "Description"}
    )
    for item in data
]

# Now you can safely pass it to the splitter
splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = splitter.split_documents(documents)



embeddings = OpenAIEmbeddings()

db = Chroma.from_documents(split_docs, embeddings)
query = user_pref
query_old = """
    Based on the list of similar listings that are available for sale in the context, tell me what I should choose
    considering that my faamily consists of myself, my wife and two daughters. A good school is important to me and 
    away from hustle and bustle of city. Please also list the details of the property you are recommending. Also print the 
    metadata for this property.
    """

use_chain_helper = False
if use_chain_helper:
    rag = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever())
    print(rag.run(query))
else:
    similar_docs = db.similarity_search(query, k=5)
    prompt = PromptTemplate(
        template="{query}\nContext: {context}",
        input_variables=["query", "context"],
    )
    chain = load_qa_chain(llm, prompt = prompt, chain_type="stuff")
    print(chain.run(input_documents=similar_docs, query = query))

This contemporary bi-level 4-bedroom, 2-bathroom home offers comfort and sophistication with red oak hardwood flooring and recent upgrades like a new roof, boiler, and sanitary system. The large living room provides ample space for holiday gatherings, and three freshly stained multi-level decks make entertaining easy. The spacious yard with three sheds offers plenty of outdoor storage and space for outdoor activities. Located in Chappaqua P.O. within the prestigious Pleasantville School system, this home also offers the convenience of a rare free school bus pick up and is within walking distance to Pleasantville schools. Situated on a cul-de-sac, this home is just a 5-minute walk to the Chappaqua Train station, providing easy access to Grand Central Station in just 45 minutes. The park-like grounds offer freedom and adventure, while the cathedral ceilings and red oak hardwood floors inside add elegance and style. With a 1-car attached garage and proximity to farmers markets, shops, tra

In [36]:
def ask_questions():
    questions = [
        "What is your name?",
        "What type of property are you looking for?",
        "What is your budget?",
        "Which location do you prefer?",
        "Any must-have features in the house?"
    ]

    answers = []

    print("Please answer the following questions:\n")

    for question in questions:
        answer = input(f"{question} ")
        answers.append(answer)

    print("\n✅ Data collection complete.\n")

    return questions, answers

# Run the Q&A collection
questions, answers = ask_questions()

# Display results
print("📝 Questions List:")
print(questions)

print("\n💬 Answers List:")
print(answers)


Please answer the following questions:

What is your name? Sudip
What type of property are you looking for? Clonial
What is your budget? 100000
Which location do you prefer? Scarsdale
Any must-have features in the house? Everything

✅ Data collection complete.

📝 Questions List:
['What is your name?', 'What type of property are you looking for?', 'What is your budget?', 'Which location do you prefer?', 'Any must-have features in the house?']

💬 Answers List:
['Sudip', 'Clonial', '100000', 'Scarsdale', 'Everything']
